In [2]:
import os
import pandas as pd

df = pd.read_csv(os.environ.get('FINETUNE_CSV_PATH'))

In [3]:
df = df.T.reset_index().T
new_columns = ['instruction', 'output']
df.columns = new_columns
df.index = range(0, len(df))
df.head()

,instruction,output
0,find the first name and last name of entities ...,"SELECT FIRST_NAME, LAST_NAME FROM JPK_ENTITY W..."
1,show all headers sent after 1st January 2023,SELECT * FROM JPK_HEADER WHERE SEND_TIME > '20...
2,get all supplier numbers who have a purchase d...,SELECT SUPPLIER_NUMBER FROM VAT_PURCHASE WHERE...
3,fetch all purchases from supplier number 12345,SELECT * FROM VAT_PURCHASE WHERE SUPPLIER_NUMB...
4,list sales between April and July 2023,SELECT * FROM VAT_SALE WHERE ISSUE_DATE BETWEE...


In [4]:
import re

def rewrite_DDL(DDL):
    lines = DDL.split('\n')

    modified_text = "DDL: "
    for line in lines:
        if len(line) < 2:
            pass
        elif '/*' in line:
            pass
        elif 'DROP TABLE' in line:
            pass

        
        else:
            line = re.sub(r'(?<!NOT )\bNULL\b', '', line)
            modified_text += f'{line}\n'

    
    return modified_text

with open(f"{os.environ.get('SQL_FILES_PATH')}/ddl.sql", "r") as file:
    ddl = file.read()

rewriten_ddl = rewrite_DDL(ddl)
df['input'] = rewriten_ddl

"DDL: CREATE TABLE 'JPK_HEADER'\n    'HEADER_ID' NUMERIC NOT NULL PRIMARY KEY,\n    'SEND_TIME' NUMERIC NOT NULL,\n    'CREATION_TIME' NUMERIC NOT NULL,\n    'DATE_FROM' NUMERIC NOT NULL,\n    'DATE_TO' NUMERIC ,\n    'YEAR_MONTH' NUMERIC NOT NULL\n);\nCREATE TABLE 'JPK_ENTITY'\n    'ENTITY_ID' NUMERIC NOT NULL PRIMARY KEY,\n    'HEADER_ID' NUMERIC NOT NULL,\n    'TAX_ID' TEXT NOT NULL,\n    'FIRST_NAME' TEXT NOT NULL,\n    'LAST_NAME' TEXT NOT NULL,\n    'BIRTH_DATE' NUMERIC NOT NULL,\n    'PHONE' NUMERIC ,\n    CONSTRAINT 'FK_JPK_ENTITY_JPK_HEADER' FOREIGN KEY ('HEADER_ID') REFERENCES 'JPK_HEADER' ('HEADER_ID') ON DELETE No Action ON UPDATE No Action\n);\nCREATE TABLE 'VAT_SALE'\n    'SALE_ID' NUMERIC NOT NULL PRIMARY KEY,\n    'HEADER_ID' NUMERIC NOT NULL,\n    'CONTRACTOR_NUMBER' TEXT NOT NULL,\n    'SALES_DOCUMENT' TEXT NOT NULL,\n    'ISSUE_DATE' NUMERIC NOT NULL,\n    'SALE_DATE' NUMERIC NOT NULL,\n    'P_6' NUMERIC ,\n    'P_8' NUMERIC ,\n    'P_9' NUMERIC ,\n    'P_11' NUMERIC

In [9]:
text_col = []

for _, row in df.iterrows():
    prompt = 'Make SQLite query based on DDL and instruction.'
    instruction = row['instruction']
    input_query = row['input']
    response = row['output']

    text = (
        prompt
        + '### Instruction:\n'
        + instruction
        + '### Input:\n'
        + input_query
        + '### Response:\n'
        + response
    )

    text_col.append(text)

df['text'] = text_col

In [10]:
df.to_csv('train.csv', index=False)
# df[20:].to_csv('train.csv', index=False)
# df[:20].to_csv('test.csv', index=False)
